In [1]:
# import libraries

import openeo
import folium
import json
import shapely.geometry
from openeo.processes import ProcessBuilder
import os


# connect to openeo
conn = openeo.connect("https://openeo.vito.be").authenticate_oidc()

# out dir
outdir = "./input"

# aoi
aoi = json.load(open('input/senales.geojson'))

# check the aoi
region = aoi['features'][0]['geometry']
geom = shapely.geometry.shape(region)
centroid = geom.centroid
center_latlon = [centroid.y, centroid.x]

m = folium.Map(location=center_latlon, zoom_start=9)

folium.GeoJson(aoi).add_to(m)
m

# define time period
time_period = ['2024-04-02', '2024-04-02']

Authenticated using refresh token.


In [2]:
s2 = conn.load_collection(
    'SENTINEL2_L1C',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=["B02", "B03", "B04", "B08", "B11"])
 
s2_L2A = conn.load_collection(
    'SENTINEL2_L2A',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=['SCL'])

worldcover = conn.load_collection(
    'ESA_WORLDCOVER_10M_2021_V2',
    spatial_extent=region,
    bands=['MAP']).resample_spatial(projection=32632)

dem = conn.load_collection(
    "COPERNICUS_30",
    spatial_extent=region,
    bands=["DEM"]).resample_spatial(projection=32632)

water_mask = (worldcover == 80).reduce_dimension(dimension="t", reducer="mean")

s2 = s2.merge_cubes(s2_L2A).merge_cubes(water_mask)

green = s2.band("B03")
swir = s2.band("B11")
nir = s2.band("B08")
scl = s2.band("SCL")
water = s2.band("MAP")

# NDSI
ndsi = (green - swir) / (green + swir)

# cloud mask
cloud_mask = ( (scl == 8) | (scl == 9) | (scl == 3) | (scl == 10) ) * 1.0 # times one forces to binary


In [3]:
valid_mask = (~cloud_mask) & (water !=1)

In [4]:
# valid_mask.download(os.path.join(outdir,'valid_mask.nc'))

In [5]:
snow_sure = (ndsi > 0.6) & (nir > 0.45) & valid_mask
no_snow_sure = (ndsi < 0) & valid_mask

In [6]:
# snow_sure.download(os.path.join(outdir,'snow_sure.nc'))
# no_snow_sure.download(os.path.join(outdir,'no_snow_sure.nc'))

In [7]:
# Combine to a snow_map: 0 = uncertain, 1 = sure no-snow, 2 = sure snow
snow_map = snow_sure.multiply(2) + no_snow_sure.multiply(1)

In [8]:
# snow_map.download(os.path.join(outdir,'snow_map.nc'))

In [11]:
# # distance 

# distance_udf = openeo.UDF.from_file("distance_udf.py")

# distance = snow_sure.apply_neighborhood(process=distance_udf,
#                                         size=[{"dimension": "x", "value": 512, "unit": "px"},
#                                               {"dimension": "y", "value": 512, "unit": "px"},],
#                                         overlap=[{"dimension": "x", "value": 256, "unit": "px"},
#                                                  {"dimension": "y", "value": 256, "unit": "px"}]) 

In [12]:
# distance = distance.save_result(format="NetCDF")
# job = distance.create_job()
# job.start_and_wait()
# job.download_results("distance/")

In [13]:
# # normalize distance to range 0f 0-1
# distance_min = distance.reduce_dimension("t", reducer="min")
# distance_max = distance.reduce_dimension("t", reducer="max")

# normalized_distance = (distance - distance_min) / (distance_max - distance_min)


In [14]:
# # normalized_distance.download('input/normalized_distance.nc')
# normalized_distance = normalized_distance.save_result(format="NetCDF")
# job = distance.create_job()
# job.start_and_wait()
# job.download_results("input/")

Here I decided to switch to single date datacube instead of the month I previously had. Basically all the problems so far where because of the multiple time steps. And now the final straw was that masking the dem with snow_sure would require the dem also to have the same time steps as the snow_sure. I tried to add them but couldn't figure it out. So only one day now instead of multiple days.

#### mask dem with snow_sure

In [15]:
snow_mask = snow_sure.reduce_dimension("t", reducer="mean") # snowmap and dem have different time values in 't', so reduce 't'
# snow_mask.download(os.path.join(outdir,'snowmask.nc'))

In [16]:
dem = dem.reduce_dimension(dimension='t', reducer='mean')
# dem.download(os.path.join(outdir,'dem.nc'))

In [17]:
masked_dem = dem.mask(snow_sure==0) # get dem for confident snow areas only
# masked_dem.download(os.path.join(outdir,'masked_dem.nc'))

In [18]:
# print(masked_dem.metadata)

#### combine snow_sure and dem to get altitude_mask

In [21]:
# snow_sure = snow_sure.add_dimension(name="bands", label="snow_sure", type="bands")
# dem = dem.rename_labels("bands", ["DEM"]) 
# combined = dem.merge_cubes(snow_sure)

# combined = dem.rename_labels("bands", ["DEM"]).merge_cubes(
#     masked_dem.rename_labels("bands", ["masked_DEM"])
# )


In [22]:
# print(combined.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180.0013889, -89.9998611, 179.9986111, 84.0001389]]}, 'temporal': {'interval': [['2010-12-12T00:00:00Z', None]]}} - ['DEM', 'snow_sure'] - ['bands', 't', 'x', 'y'])


In [52]:
## this one runs but gives boxy look due to chunking

# altitude_mask_udf = openeo.UDF.from_file("altitude_mask_udf.py")

# altitude_mask = combined.apply_neighborhood(process=altitude_mask_udf,
#                                         size=[{"dimension": "x", "value": 256, "unit": "px"},
#                                               {"dimension": "y", "value": 256, "unit": "px"},],
#                                         overlap=[{"dimension": "x", "value": 128, "unit": "px"},
#                                                  {"dimension": "y", "value": 128, "unit": "px"}])

In [53]:
# altitude_mask = altitude_mask.save_result(format="NetCDF")
# job = altitude_mask.create_job()
# job.start_and_wait()
# job.download_results(outdir)